In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [2]:
spark_config = SparkConf().setMaster("local[4]")
spark_config.set("spark.yarn.dist.jars", "C:\\Users\\HP\Desktop\ojdbc8.jar")


In [3]:
sc = SparkContext(conf=spark_config) 
sqlContext = SQLContext(sc)

In [4]:
spark = SparkSession.builder.master("local").appName("SparkOracleTest").getOrCreate()

In [5]:
spark

In [ ]:
query='''SELECT * FROM CUSTOMER C, ORDERS O WHERE C.ID = O.CUSTOMER_ID'''

In [9]:
customer = spark.read.format("jdbc").option("url", "jdbc:oracle:thin:@192.168.217.202:1521/KOPODA")\
.option("dbtable", "customer").\
option("user", "DA2103").\
option("password", "da03").\
option("numPartitions", 200).\
option("fetchsize", 100).load()

orders = spark.read.format("jdbc").option("url", "jdbc:oracle:thin:@192.168.217.202:1521/KOPODA")\
.option("dbtable", "orders").\
option("user", "DA2103").\
option("password", "da03").\
option("numPartitions", 200).\
option("fetchsize", 100).load()

In [10]:
ta = customer.alias('ta')
tb = orders.alias('tb') 

In [11]:
inner_join = ta.join(tb, ta.ID == tb.CUSTOMER_ID, "inner")

In [12]:
result = inner_join.groupBy('ACCOUNT_MGR').agg(F.avg('CREDIT_LIMIT')).orderBy('ACCOUNT_MGR')

In [ ]:
import time
start = time.time()
result.show()
print("time :", time.time() - start)

In [21]:
result

DataFrame[ID: string, NAME: string, ZIPCODE: string, ADDRESS1: string, ADDRESS2: string]

In [23]:
result.length("ID")

AttributeError: 'DataFrame' object has no attribute 'length'

In [17]:
import time
start = time.time()

result.write \
    .format("jdbc") \
    .option("url", "jdbc:oracle:thin:@192.168.217.202:1521/KOPODA")\
    .option("dbtable", "DA2103.spark_result") \
    .option("user", "DA2103") \
    .option("password", "da03") \
    .option("batchsize",10000)\
    .save()


print("time :", time.time() - start)

Py4JJavaError: An error occurred while calling o142.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 11.0 failed 1 times, most recent failure: Lost task 3.0 in stage 11.0 (TID 620) (DESKTOP-M2CTA8K executor driver): java.sql.BatchUpdateException: ORA-01653: DA2103.SPARK_RESULT 테이블을 128(으)로 USERS 테이블스페이스에서 확장할 수 없습니다

	at oracle.jdbc.driver.OraclePreparedStatement.executeLargeBatch(OraclePreparedStatement.java:10032)
	at oracle.jdbc.driver.T4CPreparedStatement.executeLargeBatch(T4CPreparedStatement.java:1364)
	at oracle.jdbc.driver.OraclePreparedStatement.executeBatch(OraclePreparedStatement.java:9839)
	at oracle.jdbc.driver.OracleStatementWrapper.executeBatch(OracleStatementWrapper.java:234)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:687)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:856)
	at org.apache.spark.sql.execution.datasources.